# IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import nltk
import gensim
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
from gensim import corpora, models
import re
stemmed_tokens=[]

# IMPORT DATA

In [2]:
df=pd.read_csv('news_articles.csv')
df=df[['Article_Id','Title','Content']].dropna()
doc_set=list(df['Content'])

# DATA PRE-PROCESSING

In [3]:
for i in range(4831):
    raw=doc_set[i].lower()
    raw=re.sub(r'\d+', '', raw)
    word_tokens = tokenizer.tokenize(raw)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    lemmatised_words=[lemmatizer.lemmatize(w) for w in filtered_sentence]
    stemmed_words=[stemmer.stem(i) for i in lemmatised_words]
    stemmed_tokens.append(stemmed_words)   

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/nishant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# LDA MODEL

In [5]:
dictionary=corpora.Dictionary(stemmed_tokens)
doc_term_matrix=[dictionary.doc2bow(d) for d in stemmed_tokens]

In [6]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=20,minimum_probability=0.0)

In [7]:
#ldamodel=gensim.models.ldamodel.LdaModel(doc_term_matrix,num_topics=5,id2word=dictionary,passes=20,minimum_probability=0.0)

In [8]:
item=ldamodel[doc_term_matrix[1]]
item

[(0, 0.0017065796),
 (1, 0.001731139),
 (2, 0.49652022),
 (3, 0.49834147),
 (4, 0.001700563)]

In [250]:
topics=[]
for i in range(4831):
    item=ldamodel[doc_term_matrix[i]]
    item=np.array(item).T
    item=item[1]
    topics.append(item) 

np.shape(topics)


(4831, 5)

In [10]:
def get_lda_topics(model,num_topics):
    word_dict={};
    for i in range(num_topics):
        words=model.show_topic(i,topn=10);
        word_dict['Topic#' + '{:02d}'.format(i+1)]=[i[0] for i in words]
    return pd.DataFrame(word_dict);

In [11]:
get_lda_topics(ldamodel,5)

,Topic#01,Topic#02,Topic#03,Topic#04,Topic#05
0,step,india,said,film,featur
1,instal,year,report,movi,smartphon
2,android,said,polic,india,gb
3,custom,r,state,also,devic
4,rom,price,also,khan,camera
5,user,govern,minist,salman,also
6,devic,market,attack,actor,g
7,note,crore,peopl,year,new
8,recoveri,compani,govern,releas,galaxi
9,googl,bank,india,one,xiaomi


In [12]:
topics=np.array(topics)
np.shape(topics)

(4831, 5)

# K-Means CLUSTERING

In [135]:
import sklearn
from sklearn.cluster import KMeans
kmeans=sklearn.cluster.KMeans(5).fit(topics)
clusters=kmeans.labels_

In [199]:
clusters

array([2, 1, 2, ..., 4, 4, 4])

# User Profile

In [149]:
df=pd.read_csv('news_articles.csv')
df=df[['Article_Id','Title','Content']].dropna()
content=list(df['Content'])
title=list(df["Title"])

length_article=([len(x) for x in df['Content']] ) #length of articles
user_speed=5
expected_time=[int((x/5)) for x in length_article]# expected time

possion = np.random.poisson(5, 4831)/10 

actual_time=expected_time*possion # 




        
        

In [245]:
expected_time1=np.array(expected_time)
ratio=actual_time.reshape(4831,1)/expected_time1.reshape(4831,1)
ratio=ratio.reshape(4831)
click_through= np.random.binomial(1, .8, 4831).reshape(4831)

In [253]:
data=pd.DataFrame({"Title":title,"expected_time":expected_time,"actual_time":actual_time,"ratio":ratio,"click":click_through})

In [254]:
user_prof=pd.DataFrame()
for i in range(5):
    user=data[clusters==i][:1]   
    user_prof=user_prof.append(user)

In [259]:
topics=np.array(topics)
topics=topics.reshape(4831,5)

In [262]:
topics[388]

array([0.59342319, 0.10164069, 0.10164056, 0.10164658, 0.10164895])